We showed that the VGG-16 network can be used to encode images it was not trained on---a cheese grater and a foot file---as vectors that are perfectly recognized by K nearest neighbors. What do you think would happen if we used nearest neighbors (or naive Bayes or a neural network) classifier on the raw vectors?

We'll download addtional images from GitHub below

In [ ]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tools.zip
!unzip tools.zip

--2020-07-02 21:22:21--  https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tools.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/tools.zip [following]
--2020-07-02 21:22:21--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/tools.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2502460 (2.4M) [application/zip]
Saving to: ‘tools.zip’

tools.zip           100%[===================>]   2.39M  10.3MB/s    in 0.2s    

2020-07-02 21:22:22 (10.3 MB/s) - ‘tools.zip’ saved [2502460/2502460]

Archive:  tools.zip
  inflating: 

Loading the raw image vectors as a dataset.

In [ ]:
# get images
from PIL import Image
from IPython.display import display
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
         
dat = []
labs = []
imgs = []
imgflist = ["cg01", "cg02", "cg03", "cg04", "cg05", "cg06", "cg07", "cg08", "cg09", "cg10",
           "ff01", "ff02", "ff03", "ff04", "ff05", "ff06", "ff07", "ff08", "ff09", "ff10"]
for imgf in imgflist:
    img = image.load_img(imgf+".jpeg", target_size=(224,224))
    imgs.append(img)
    img_arr = np.expand_dims(image.img_to_array(img), axis=0)
#    x = preprocess_input(img_arr)
#    preds = model2.predict(x)
    dat.append(img_arr.flatten())
    labs.append(imgf[0:2] == 'ff')

Dividing the data into training and testing data.

In [ ]:
train = [i for i in range(0,5)]+[i for i in range(10,15)]
test = [i for i in range(5,10)]+[i for i in range(15,20)]

X_train = [dat[inst] for inst in train]
y_train = [labs[inst] for inst in train]
X_test =  [dat[inst] for inst in test]
y_test =  [labs[inst] for inst in test]

Train using Naive Bayes.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

0.5


Train using nearest neighbor.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=1,metric="cosine")
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

0.4


Train using neural network.

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=[100, 100, 100, 100, 100], max_iter = 10000)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

0.5


For comparison, here are those same algorithms using the representation that comes from VGG16.

In [ ]:
!pip install keras=='2.3.1'
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K

model = vgg16.VGG16(weights='imagenet', include_top=True)

model2 = Model(model.input, model.layers[-2].output)

553467904/553467096 [==============================] - 6s 0us/step


Gather the vectors.

In [ ]:
datraw = dat
dat = []

for img in imgs:
    img_arr = np.expand_dims(image.img_to_array(img), axis=0)
    x = preprocess_input(img_arr)
    preds = model2.predict(x)
    dat.append(preds[0])

Make a new training/testing set.

In [ ]:
train = [i for i in range(0,5)]+[i for i in range(10,15)]
test = [i for i in range(5,10)]+[i for i in range(15,20)]

X_train = [dat[inst] for inst in train]
y_train = [labs[inst] for inst in train]
X_test =  [dat[inst] for inst in test]
y_test =  [labs[inst] for inst in test]

In [ ]:
Naive Bayes.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

1.0


Nearest neighbors.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=1,metric="cosine")
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

1.0


And neural networks, all perfect.

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=[100, 100, 100, 100, 100], max_iter = 10000)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

print(score)

1.0
